---
**License**

 xcdm_nopert

 Thu Mar 16 16:56:00 2023\
 Copyright  2023\
 Sandro Dias Pinto Vitenti  <vitenti@uel.br>

---
---

 xcdm_nopert\
 Copyright (C) 2023 Sandro Dias Pinto Vitenti  <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
import sys

from numcosmo_py import Ncm

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import numpy as np
import math

import getdist
from getdist import plots

from numcosmo_py.experiments.xcdm_no_perturbations import run_xcdm_nopert_mcmc

from numcosmo_py.plotting.tools import confidence_ellipse
from numcosmo_py.plotting.tools import set_rc_params_article
from numcosmo_py.plotting.getdist import mcat_to_mcsamples
from numcosmo_py.sampling.esmcmc import mcat_print_info, WalkerTypes

In [ ]:
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

ssize = 2000000
nthreads = 4
nwalkers_tiny = 64
nwalkers_small = 600
nwalkers = 2000
thin = 1
use_neutrino = True
save_figs = True
verbose = False

Ncm.func_eval_set_max_threads(nthreads)
Ncm.func_eval_log_pool_stats()

In [ ]:
catalog_apes_small = run_xcdm_nopert_mcmc(
    sampler=WalkerTypes.APES,
    nwalkers=nwalkers_small,
    ssize=ssize,
    verbose=verbose,
    nthreads=nthreads,
    use_neutrino=use_neutrino,
)
catalog_apes = run_xcdm_nopert_mcmc(
    sampler=WalkerTypes.APES,
    nwalkers=nwalkers,
    ssize=ssize,
    verbose=verbose,
    nthreads=nthreads,
    use_neutrino=use_neutrino,
)
catalog_stretch_tiny = run_xcdm_nopert_mcmc(
    sampler=WalkerTypes.STRETCH,
    nwalkers=nwalkers_tiny,
    ssize=ssize,
    verbose=verbose,
    nthreads=nthreads,
    use_neutrino=use_neutrino,
)
catalog_stretch_small = run_xcdm_nopert_mcmc(
    sampler=WalkerTypes.STRETCH,
    nwalkers=nwalkers_small,
    ssize=ssize,
    verbose=verbose,
    nthreads=nthreads,
    use_neutrino=use_neutrino,
)
catalog_stretch = run_xcdm_nopert_mcmc(
    sampler=WalkerTypes.STRETCH,
    nwalkers=nwalkers,
    ssize=ssize,
    verbose=verbose,
    nthreads=nthreads,
    use_neutrino=use_neutrino,
)

In [ ]:
def load_cat_skip_burnin(catalog):
    mcat = Ncm.MSetCatalog.new_from_file_ro(catalog, 0)
    max_t, _ = mcat.calc_max_ess_time(100, Ncm.FitRunMsgs.SIMPLE)
    return mcat, max_t * mcat.nchains()


mcat_apes_small, apes_burnin_small = load_cat_skip_burnin(catalog_apes_small)
mcat_apes, apes_burnin = load_cat_skip_burnin(catalog_apes)

mcat_stretch_tiny, stretch_burnin_tiny = load_cat_skip_burnin(catalog_stretch_tiny)
mcat_stretch_small, stretch_burnin_small = load_cat_skip_burnin(catalog_stretch_small)
mcat_stretch, stretch_burnin = load_cat_skip_burnin(catalog_stretch)

param1 = 0
param2 = 4
param1_symbol = mcat_apes.col_symb(param1 + mcat_apes.nadd_vals())
param2_symbol = mcat_apes.col_symb(param2 + mcat_apes.nadd_vals())

assert mcat_apes_small.nchains() == nwalkers_small
assert mcat_apes.nchains() == nwalkers
assert mcat_stretch_tiny.nchains() == nwalkers_tiny
assert mcat_stretch_small.nchains() == nwalkers_small
assert mcat_stretch.nchains() == nwalkers

In [ ]:
sample_apes_small, rows_apes_small, posterior_apes_small = mcat_to_mcsamples(
    mcat_apes_small,
    "APES",
    thin=thin,
    burnin=apes_burnin_small,
)
del mcat_apes_small

sample_apes, rows_apes, posterior_apes = mcat_to_mcsamples(
    mcat_apes,
    "APES",
    thin=thin,
    burnin=apes_burnin,
)
del mcat_apes

sample_stretch_tiny, rows_stretch_tiny, posterior_stretch_tiny = mcat_to_mcsamples(
    mcat_stretch_tiny,
    "STRETCH",
    thin=thin,
    burnin=stretch_burnin_tiny,
)
del mcat_stretch_tiny

sample_stretch_small, rows_stretch_small, posterior_stretch_small = mcat_to_mcsamples(
    mcat_stretch_small,
    "STRETCH",
    thin=thin,
    burnin=stretch_burnin_small,
)
del mcat_stretch_small

sample_stretch, rows_stretch, posterior_stretch = mcat_to_mcsamples(
    mcat_stretch,
    "STRETCH",
    thin=thin,
    burnin=stretch_burnin,
)
del mcat_stretch

In [ ]:
print(
    sample_apes_small.getConvergeTests(
        what=("MeanVar", "GelmanRubin", "SplitTest", "CorrLengths"), feedback=True
    )
)
print(
    sample_apes.getConvergeTests(
        what=("MeanVar", "GelmanRubin", "SplitTest", "CorrLengths"), feedback=True
    )
)
print(
    sample_stretch_tiny.getConvergeTests(
        what=("MeanVar", "GelmanRubin", "SplitTest", "CorrLengths"), feedback=True
    )
)
print(
    sample_stretch_small.getConvergeTests(
        what=("MeanVar", "GelmanRubin", "SplitTest", "CorrLengths"), feedback=True
    )
)
print(
    sample_stretch.getConvergeTests(
        what=("MeanVar", "GelmanRubin", "SplitTest", "CorrLengths"), feedback=True
    )
)

In [ ]:
set_rc_params_article(ncol=2)

g = getdist.plots.get_subplot_plotter(width_inch=plt.rcParams["figure.figsize"][0])
g.settings.linewidth = 0.01
g.triangle_plot([sample_apes_small, sample_stretch_small], shaded=True)

if save_figs:
    plt.savefig("xcdm_nopert_corner.pdf", bbox_inches="tight")

In [ ]:
set_rc_params_article(ncol=2)

fig, ax = plt.subplots()

ax.set(xlim=(67, 77), ylim=(0, 0.4))
ax.set_xlabel(f"${param1_symbol}$")
ax.set_ylabel(f"${param2_symbol}$")

scat_apes_small = ax.scatter(
    rows_apes_small[0:nwalkers_small, param1],
    rows_apes_small[0:nwalkers_small, param2],
    s=4,
    c="red",
)
scat_stretch_small = ax.scatter(
    rows_stretch_small[0:nwalkers_small, param1],
    rows_stretch_small[0:nwalkers_small, param2],
    s=4,
    c="blue",
)

nframes = 600
last_t = int(
    min(2 * max(stretch_burnin_small, apes_burnin_small), rows_apes_small.shape[0])
    / nwalkers_small
)
if last_t > nframes:
    step = last_t // nframes
    b = range(0, last_t, step)
else:
    b = range(last_r)


def animate(i):
    x_i = rows_apes_small[nwalkers_small * b[i] : nwalkers_small * b[i + 1], param1]
    y_i = rows_apes_small[nwalkers_small * b[i] : nwalkers_small * b[i + 1], param2]
    x2_i = rows_stretch_small[nwalkers_small * b[i] : nwalkers_small * b[i + 1], param1]
    y2_i = rows_stretch_small[nwalkers_small * b[i] : nwalkers_small * b[i + 1], param2]
    scat_apes_small.set_offsets(np.c_[x_i, y_i])
    scat_stretch_small.set_offsets(np.c_[x2_i, y2_i])


anim = FuncAnimation(fig, animate, interval=20, frames=nframes - 1)
pass

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
writervideo = animation.FFMpegWriter(fps=20)
anim.save("xcdm_no_pert.mp4", writer=writervideo)
plt.close()

In [ ]:
set_rc_params_article(ncol=2)

fig = plt.figure()
gs = fig.add_gridspec(2, hspace=0)
axs = gs.subplots(sharex=True)
fig.suptitle("Parameter evolution")

step = 1
alpha = 0.6

t_apes = np.arange(last_t, step=step)
t_stretch = np.arange(last_t, step=step)

last_p = last_t * nwalkers_small

for i in range(0, nwalkers_small, 30):
    x_1_apes_small = rows_apes_small[i:last_p:nwalkers_small, param1]
    x_1_stretch_small = rows_stretch_small[i:last_p:nwalkers_small, param1]
    x_2_apes_small = rows_apes_small[i:last_p:nwalkers_small, param2]
    x_2_stretch_small = rows_stretch_small[i:last_p:nwalkers_small, param2]

    axs[0].scatter(t_apes, x_1_apes_small[::step], s=0.1, alpha=alpha, color="r")
    axs[0].scatter(t_stretch, x_1_stretch_small[::step], s=0.1, alpha=alpha, color="b")

    axs[1].scatter(t_apes, x_2_apes_small[::step], s=0.1, alpha=alpha, color="r")
    axs[1].scatter(t_stretch, x_2_stretch_small[::step], s=0.1, alpha=alpha, color="b")

# axs[1].set_yscale("symlog")

axs[0].set_ylabel(f"${param1_symbol}$")
axs[1].set_ylabel(f"${param2_symbol}$")
axs[1].set_xlabel("$t$")

if save_figs:
    plt.tight_layout()
    plt.savefig("xcdm_nopert_param_evol.pdf", bbox_inches="tight")

pass

In [ ]:
kernel = Ncm.StatsDistKernelST.new(6, 1.0)
interp_vkde = Ncm.StatsDistVKDE.new(kernel, Ncm.StatsDistCV.NONE)
interp_kde = Ncm.StatsDistKDE.new(kernel, Ncm.StatsDistCV.NONE)

# interp_vkde.set_cov_type(Ncm.StatsDistKDECovType.ROBUST)
# interp_kde.set_cov_type(Ncm.StatsDistKDECovType.ROBUST)

In [ ]:
max_n = len(rows_apes_small)
ssize = int(nwalkers / 2)

interp_vkde.reset()
interp_kde.reset()

for theta in rows_apes_small[-ssize:]:
    theta_v = Ncm.Vector.new_array(theta)
    interp_vkde.add_obs(theta_v)
    interp_kde.add_obs(theta_v)

m2lnL_v = Ncm.Vector.new_array(2.0 * posterior_apes_small[-ssize:])

interp_vkde.prepare_interp(m2lnL_v)
interp_kde.prepare_interp(m2lnL_v)

In [ ]:
set_rc_params_article(ncol=1)
fig, ax = plt.subplots(1, 1)

indices = [param1, param2]

for i in range(interp_vkde.get_sample_size()):
    y_i, cov_i, n_i, w_i = interp_vkde.get_Ki(i)
    mu = np.array(y_i.dup_array())
    cov = np.array([[cov_i.get(i, j) for j in indices] for i in indices])
    cov = cov * 1.0

    confidence_ellipse(mu[indices], cov, ax, edgecolor="red")

ax.set_xlabel(f"${param1_symbol}$")
ax.set_ylabel(f"${param2_symbol}$")
ax.autoscale_view()
plt.grid()

if save_figs:
    plt.tight_layout()
    plt.savefig("xcdm_nopert_vkde_kernels.pdf", bbox_inches="tight")

In [ ]:
set_rc_params_article(ncol=1)
fig, ax = plt.subplots(1, 1)

for i in range(interp_kde.get_sample_size()):
    y_i, cov_i, n_i, w_i = interp_kde.get_Ki(i)
    mu = np.array(y_i.dup_array())
    cov = np.array([[cov_i.get(i, j) for j in indices] for i in indices])
    cov = cov * 1.0

    confidence_ellipse(mu[indices], cov, ax, edgecolor="red")

ax.set_xlabel(f"${param1_symbol}$")
ax.set_ylabel(f"${param2_symbol}$")
ax.autoscale_view()
plt.grid()

if save_figs:
    plt.tight_layout()
    plt.savefig("xcdm_nopert_kde_kernels.pdf", bbox_inches="tight")